<a href="https://colab.research.google.com/github/mattos21/-DeeperTask/blob/master/CNN_train_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install PyDrive

In [0]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
download = drive.CreateFile({'id': '1lLQzJAkIbAWOGd9WuxmBB22NVWFCjOEk'})
download.GetContentFile('dataset.zip')

download = drive.CreateFile({'id': '1uSI3jsbroZE3PcmEbWkdmBvGqMWOH4OA'})
download.GetContentFile('validate.zip')

!unzip dataset.zip
!unzip validate.zip

In [0]:
download = drive.CreateFile({'id': '18lS5VWdO6D6Ow4NcYAkAZ6ADVJnm8vE3'})
download.GetContentFile('top.h5')

In [7]:
!ls

adc.json     sample_data  train_dataset  validate.zip
dataset.zip  top.h5	  validate


In [9]:
'''
In summary, this is our directory structure:
```
data/
    train/
        rotated_left/
            001.jpg
            002.jpg
            ...
        rotated_right/
            001.jpg
            002.jpg
            ...
        
```
'''

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import Flatten, Dense, Dropout
from keras import backend as K

import os

from keras.optimizers import SGD
img_width, img_height = 64, 64

path = os.getcwd()
train_data_dir = path + '/train_dataset'
validation_data_dir = path + '/validate'
nb_train_samples = 40001
#epochs = 250
epochs = 80
batch_size = 120

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

# create model
model = Sequential()

# add model layers
model.add(Conv2D(64, kernel_size=5, activation='relu', input_shape=input_shape))
model.add(Conv2D(32, kernel_size=5, activation='relu'))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))

# compile model
#opt = SGD(lr=0.035)

#sgd = SGD(lr=0.035, decay=1e-6)
sgd = SGD(lr=0.005, decay=1e-6)

model.compile(loss = "categorical_crossentropy", optimizer = sgd, metrics=['acc'])

#train_datagen = ImageDataGenerator()
model.load_weights(path + "/top.h5")

test_datagen = ImageDataGenerator(rescale=1./255)

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
      train_data_dir,
      target_size=(img_width, img_height),
      batch_size=batch_size,
      class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(64, 64),
        batch_size=batch_size,
        class_mode='categorical')

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps= 8000// batch_size)
    

model.save_weights('top.h5')

Found 40001 images belonging to 4 classes.
Found 8895 images belonging to 4 classes.
Epoch 1/80
333/333 [==============================] - 76s 229ms/step - loss: 0.1391 - acc: 0.9386 - val_loss: 0.6314 - val_acc: 0.7668
Epoch 2/80
333/333 [==============================] - 74s 223ms/step - loss: 0.1378 - acc: 0.9405 - val_loss: 0.6170 - val_acc: 0.7663
Epoch 3/80
333/333 [==============================] - 73s 221ms/step - loss: 0.1392 - acc: 0.9385 - val_loss: 0.6413 - val_acc: 0.7606
Epoch 4/80
333/333 [==============================] - 74s 223ms/step - loss: 0.1300 - acc: 0.9437 - val_loss: 0.6451 - val_acc: 0.7631
Epoch 5/80
333/333 [==============================] - 73s 221ms/step - loss: 0.1340 - acc: 0.9406 - val_loss: 0.6700 - val_acc: 0.7605
Epoch 6/80
333/333 [==============================] - 74s 222ms/step - loss: 0.1325 - acc: 0.9424 - val_loss: 0.6792 - val_acc: 0.7546
Epoch 7/80
333/333 [==============================] - 73s 220ms/step - loss: 0.1310 - acc: 0.9422 - val_l

In [0]:

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

upload = drive.CreateFile({'title': 'top1.h5'})
upload.SetContentFile('top.h5')
upload.Upload()